In [36]:
import pandas as pd
from IPython.display import clear_output
import pandas as pd

In [37]:
import time
import random

In [38]:
name_info = pd.read_parquet('./2more_name_info_dtypes.parquet')
name_info.head(2)

Name School  Power             Link Hometown
0   Tiago ASAKAWA   UMBC   -1.0  /swimmer/515075         
1  Niklas Weigelt   UMBC   -1.0  /swimmer/413265

In [39]:
name_info.Link.value_counts()

Link
                    3075
NoMatch              298
/swimmer/394628        5
/swimmer/1329908       5
/swimmer/467284        5
                    ... 
/swimmer/1198471       1
/swimmer/401636        1
/swimmer/368115        1
/swimmer/505222        1
/swimmer/813110        1
Name: count, Length: 11508, dtype: Int64

In [40]:
name_info.shape

(16085, 5)

In [41]:
name_info = name_info[~name_info['Link'].isin(['', 'NoMatch'])]
name_info.shape

(12712, 5)

In [42]:
name_info.Link.value_counts()

Link
/swimmer/1329908    5
/swimmer/467284     5
/swimmer/394628     5
/swimmer/526334     4
/swimmer/635415     4
                   ..
/swimmer/570497     1
/swimmer/1309365    1
/swimmer/679865     1
/swimmer/1160815    1
/swimmer/813110     1
Name: count, Length: 11506, dtype: Int64

In [43]:
name_info['ID'] = name_info['Name'].str.cat(name_info['School'], sep='|')
name_info = name_info.drop_duplicates(subset='ID')
name_info.head()

Name                              School  Power             Link  \
0   Tiago ASAKAWA                                UMBC   -1.0  /swimmer/515075   
1  Niklas Weigelt                                UMBC   -1.0  /swimmer/413265   
2     Ben Beldner                          Binghamton   -1.0  /swimmer/394588   
3     Justin Meyn                          Binghamton   -1.0  /swimmer/471360   
4   Joshua Franco  New Jersey Institute of Technology   -1.0  /swimmer/395308   

  Hometown                                                ID  
0                                         Tiago ASAKAWA|UMBC  
1                                        Niklas Weigelt|UMBC  
2                                     Ben Beldner|Binghamton  
3                                     Justin Meyn|Binghamton  
4           Joshua Franco|New Jersey Institute of Technology

In [44]:
name_info = name_info.drop('ID',axis=1)

In [45]:
name_info[name_info.Name=="Vineet Ranade"]

Name                               School  Power  \
5943  Vineet Ranade  Rose-Hulman Institute of Technology   -1.0   

                 Link Hometown  
5943  /swimmer/418728

In [46]:
name_info.shape

(12698, 5)

In [47]:
name_info[name_info.groupby('Name')['Name'].transform('count') > 1]

/tmp/ipykernel_1149705/3941025534.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  name_info[name_info.groupby('Name')['Name'].transform('count') > 1]


Name                               School   Power  \
33       Daniel Jablonski   New Jersey Institute of Technology   33.73   
36            Alex Watson                                Maine   -1.00   
70          Josh Fountain          Florida Atlantic University   -1.00   
72           Connor Doyle          Virginia Military Institute    2.51   
73           Zach Bennett              Gardner-Webb University   -1.00   
...                   ...                                  ...     ...   
16079          Grace King                            Augustana   -1.00   
16081        Maddie Sauro                               Alfred  100.00   
16082  Aidah Shuttleworth                              Hanover   -1.00   
16083       Brianna Brunk  Rose-Hulman Institute of Technology   -1.00   
16084        Caroline Lee                                Wells   27.04   

                   Link Hometown  
33      /swimmer/701079           
36      /swimmer/187922           
70      /swimmer/322110           
72      /swimmer/361874           
73      /swimmer/601986           
...                 ...      ...  
16079  /swimmer/1523280           
16081  /swimmer/1573776           
16082   /swimmer/863241           
16083  /swimmer/1309404           
16084  /swimmer/1758413           

[2252 rows x 5 columns]

In [48]:
name_info.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12698 entries, 0 to 16084
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Name      12698 non-null  category
 1   School    12698 non-null  category
 2   Power     12698 non-null  float64 
 3   Link      12698 non-null  string  
 4   Hometown  12698 non-null  string  
dtypes: category(2), float64(1), string(2)
memory usage: 1.8 MB


In [49]:
name_info.to_parquet('3more_name_info_dtypes.parquet',index=False)

In [16]:
import requests
import csv
from bs4 import BeautifulSoup as bs
import pandas as pd
import time as _time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
import re

def getpi(swimmer_name):
    swimmer_name_url = 'https://swimcloud.com/recruiting/rankings/?name='+swimmer_name.replace(' ','+')
    name_url = requests.get(swimmer_name_url, headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36', 'Referer' : 'https://google.com/'})
    name_url.encoding = 'utf-8'
    name_soup = bs(name_url.text, 'html.parser')
    # print(name_soup)
    # print(name_soup)
    # print(name_url.text)
    names=[]
    links=[]
    idxs=[]
    teams=[]
    if (name_soup.find('tbody') == None):
        # print('No tbody')
        return None
    swimmer_list = name_soup.find('tbody').find_all('tr')
    i=0
    for swimmer in swimmer_list:
        if len(name_soup.find_all('td', {'class' : 'u-text-end'})) == 0:
            # print('No idx found')
            break
        name = swimmer.find(class_='u-text-semi')
        name_str = str(name)
        start = name_str.find('>') + 1
        end = name_str.find('</a>')
        # Extract the text between '>' and '<'
        name_raw = name_str[start:end]
        if (name_raw.lower() == swimmer_name.lower() or name_raw.split(' ')[-1].lower() == swimmer_name.lower()):
            names.append(name_raw)
            swimmer_link = name['href'] if name else ''
            links.append(swimmer_link)
            # if (swimmer_name == 'Reese Samuel'):
            #     print(i)
            #     print(name_soup.find_all('td', {'class' : 'u-text-end'})[i])
            if (len(name_soup.find_all('td', {'class' : 'u-text-end'})) == 0):
                return None
            idxs.append(float(name_soup.find_all('td', {'class' : 'u-text-end'})[i].text.strip()))
            team_td = swimmer.find_all('td', class_='u-text-center hidden-xs')[0]
            # Find the <img> tag within the <td>
            img_tag = team_td.find('img')
            if img_tag == None:
                # teams.append('')
                names.pop()
                links.pop()
                idxs.pop()
                # print('None team')
            else:
                # Extract the 'alt' attribute value, which contains the team name
                team_name = img_tag['alt'][:-5]
                teams.append(team_name)
        i = i + 1
    # print(names)
    # print(links)
    # print(hometowns)
    return [names,links,idxs,teams,hometowns]


In [24]:
def getPowerIndex(swimmer_ID):
	swimmer_url = 'https://swimcloud.com/swimmer/' + str(swimmer_ID)

	url = requests.get(swimmer_url, headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36', 'Referer' : 'https://google.com/'})
	url.encoding = 'utf-8'
	
	if url.status_code == 404:
		raise ValueError(f"The swimmer {swimmer_ID} was not found")

	soup = bs(url.text, 'html.parser')
	

	data_array = soup.find_all('li', {'class' : 'c-list-bar__item'}) # returns the container items containing headers for recruiting stats
	for d in data_array:
		if d.find(class_='c-list-bar__subheader').get('title') == 'Power index':
				return float(d.find(class_='c-list-bar__description').text.strip())

	swimmer_name = soup.find('h1', {'class' : 'c-toolbar__title'}).text.strip()

	swimmer_name_url = 'https://swimcloud.com/recruiting/rankings/?name=' + swimmer_name.replace(' ', '+')

	name_url = requests.get(swimmer_name_url, headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36', 'Referer' : 'https://google.com/'})

	name_url.encoding = 'utf-8'

	name_soup = bs(name_url.text, 'html.parser')

	swimmer_list = name_soup.find('tbody').find_all('tr')
	
	for swimmer in swimmer_list:
		#check if this is the correct swimmer by looking to see if the link matches their id
		name = swimmer.find(class_='u-text-semi')
		swimmer_link = name['href'] if name else ''
		if(swimmer_link == "/swimmer/" + str(swimmer_ID)):
			#a power index was found for the specified swimmer_ID!
			return float(name_soup.find('td', {'class' : 'u-text-end'}).text.strip())

	return -1 #if no swimmer is found with the correct swimmer ID #, return -1

In [35]:
import random
from IPython.display import clear_output

for i in range(len(name_info)):
    idx = row.Power
    if (idx != -1.0):
        print('Skipped')
        continue
    else:
        swimmer_ID = row.Link.split('/')[-1]
        row.Power = getPowerIndex(swimmer_ID)
        print(row.Power)
        random_number = random.randint(1, 3)
        if random_number == 1:
            clear_output()

33.13
42.86
20.61
16.29
36.41
44.47
26.76
77.97
Skipped
100.0
Skipped
Skipped
32.99
56.8
36.1
28.99
30.49
23.71
31.48
20.3


KeyboardInterrupt: 

In [ ]:
name_info.Power.value_counts()